In [34]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [59]:
import torch
import blocks
import models

In [36]:
from utils import ActNorm, Invertibe1by1Convolution
import utils

In [37]:
torch.manual_seed(42)
tmp_input = torch.randn(5, 3, 4, 4)
tmp_input[0, ...]

tensor([[[ 1.9269,  1.4873,  0.9007, -2.1055],
         [ 0.6784, -1.2345, -0.0431, -1.6047],
         [-0.7521,  1.6487, -0.3925, -1.4036],
         [-0.7279, -0.5594, -0.7688,  0.7624]],

        [[ 1.6423, -0.1596, -0.4974,  0.4396],
         [-0.7581,  1.0783,  0.8008,  1.6806],
         [ 1.2791,  1.2964,  0.6105,  1.3347],
         [-0.2316,  0.0418, -0.2516,  0.8599]],

        [[-1.3847, -0.8712, -0.2234,  1.7174],
         [ 0.3189, -0.4245,  0.3057, -0.7746],
         [-1.5576,  0.9956, -0.8798, -0.6011],
         [-1.2742,  2.1228, -1.2347, -0.4879]]])

In [38]:
ac = ActNorm(num_channels=tmp_input.shape[1])
forwarded, logdet = ac(tmp_input)

In [39]:
forwarded[0, ...]

tensor([[[ 1.7572,  1.3327,  0.7663, -2.1364],
         [ 0.5517, -1.2954, -0.1450, -1.6528],
         [-0.8296,  1.4886, -0.4823, -1.4586],
         [-0.8062, -0.6435, -0.8457,  0.6328]],

        [[ 1.5180, -0.3344, -0.6816,  0.2816],
         [-0.9497,  0.9382,  0.6529,  1.5574],
         [ 1.1446,  1.1624,  0.4572,  1.2018],
         [-0.4084, -0.1274, -0.4289,  0.7136]],

        [[-1.2794, -0.7635, -0.1126,  1.8373],
         [ 0.4322, -0.3147,  0.4190, -0.6664],
         [-1.4531,  1.1122, -0.7721, -0.4922],
         [-1.1684,  2.2446, -1.1287, -0.3784]]], grad_fn=<SelectBackward0>)

In [40]:
logdet

tensor([-0.0433, -0.0433, -0.0433, -0.0433, -0.0433],
       grad_fn=<RepeatBackward0>)

In [41]:
ac.batch_mean

Parameter containing:
tensor([[[[ 0.1071]],

         [[ 0.1657]],

         [[-0.1113]]]], requires_grad=True)

In [42]:
torch.log(ac.batch_std**2)

tensor([[[[ 0.0701]],

         [[-0.0552]],

         [[-0.0094]]]], grad_fn=<LogBackward0>)

In [43]:
torch.manual_seed(41)
inv_conv = Invertibe1by1Convolution(3)

inv_conv.weight

Parameter containing:
tensor([[[[-0.8341]],

         [[-0.0222]],

         [[-0.5512]]],


        [[[-0.1638]],

         [[-0.9441]],

         [[ 0.2860]]],


        [[[-0.5268]],

         [[ 0.3288]],

         [[ 0.7838]]]], requires_grad=True)

In [44]:
forwarded, log_det = inv_conv(tmp_input)
log_det

tensor(3.8147e-06, grad_fn=<MulBackward0>)

In [45]:
_, _, h, w = tmp_input.shape

In [46]:
h * w * torch.log(torch.abs(torch.linalg.det(inv_conv.weight.squeeze())))

tensor(3.8147e-06, grad_fn=<MulBackward0>)

In [47]:
h * w * torch.slogdet(inv_conv.weight.squeeze())[1]

tensor(3.3379e-06, grad_fn=<MulBackward0>)

In [49]:
torch.manual_seed(41)
tmp_input = torch.randn(5, 4, 4, 4)
gAff = utils.GlowAffineCouplingLayer(input_shape=(4, 4, 4), filters=3)
y, log_det = gAff(tmp_input)

In [50]:
reversed = gAff.reverse(y)
torch.linalg.norm(reversed - tmp_input)

tensor(2.8904e-07, grad_fn=<LinalgVectorNormBackward0>)

In [51]:
torch.manual_seed(41)
tmp_input = torch.randn(5, 4, 4, 4)
flow = utils.StepOfGlow(base_input_shape=tmp_input.shape[1:], num_filters=512)

In [52]:
forwarded, log_det = flow(tmp_input)

In [53]:
log_det

tensor([-2.6153, -2.6153, -2.6153, -2.6153, -2.6153], grad_fn=<AddBackward0>)

In [54]:
backwarded = flow.reverse(forwarded)

In [55]:
torch.linalg.norm(backwarded - tmp_input)

tensor(1.4738e-06, grad_fn=<LinalgVectorNormBackward0>)

In [133]:
torch.manual_seed(42)
tmp_input = torch.randn(5, 3, 4, 4)
block = blocks.GlowBlock(K=1, num_channels=3, num_filters=512, is_final_block=True)
print(list(param.numel()for param in block.parameters()))
_ = block(tmp_input)

torch.manual_seed(42)
eps = torch.distributions.Normal(0, 1).sample(sample_shape=[1, 12, 2, 2])
block.reverse(x, eps=eps)

[12, 12, 144, 27648, 512, 262144, 512, 12, 55296, 12]


/Users/amin/venvs/.tf_venv/lib/python3.9/site-packages/torch/nn/functional.py:1967: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


tensor([[[[ 0.5460, -2.8599,  0.2483,  2.1484],
          [-2.0468,  1.5575,  0.1021,  1.0958],
          [-0.3667, -0.1379, -1.3916,  0.4775],
          [-1.4393,  0.9823,  1.9102,  0.3162]],

         [[ 0.4821,  0.5251,  0.1547, -0.0842],
          [ 0.3652, -1.1456,  0.4099,  2.1670],
          [ 1.9118,  0.8759,  2.0671,  0.5911],
          [ 0.0773, -0.3874,  0.0448, -1.1593]],

         [[ 0.8032,  0.2927,  1.4028,  1.9387],
          [-0.8028, -0.0507, -0.9454, -1.1385],
          [-0.4090,  0.0237, -1.5863, -1.4707],
          [-0.2259,  0.3312,  0.5654, -1.9746]]]], grad_fn=<ViewBackward0>)

In [129]:
torch.manual_seed(42)
tmp_input = torch.randn(5, 3, 4, 4)
glow = models.Glow(K=1, L=1, base_input_shape=tmp_input.shape[1:])
glow(tmp_input)

[torch.Size([1, 12, 1, 1]), torch.Size([1, 12, 1, 1])]


(tensor([-68.0630, -61.9017, -59.3716, -64.7797, -66.3945],
        grad_fn=<AddBackward0>),
 tensor([-1.7189, -1.7189, -1.7189, -1.7189, -1.7189], grad_fn=<AddBackward0>),
 [])

In [130]:
print(list(param.numel()for param in glow.parameters()))

[12, 12, 144, 27648, 512, 262144, 512, 12, 55296, 12]


In [131]:
glow

Glow(
  (blocks): ModuleList(
    (0): GlowBlock(
      (steps): ModuleList(
        (0): StepOfGlow(
          (act_norm): ActNorm()
          (invertible_1by1_conv): Invertibe1by1Convolution()
          (aff): GlowAffineCouplingLayer(
            (relu): ReLU()
            (first_layer): Conv2d(6, 512, kernel_size=(3, 3), stride=(1, 1), padding=same)
            (second_layer): Conv2d(512, 512, kernel_size=(1, 1), stride=(1, 1), padding=same)
            (third_layer): ZeroConv2d(
              (conv): Conv2d(512, 12, kernel_size=(3, 3), stride=(1, 1))
            )
            (net): Sequential(
              (0): Conv2d(6, 512, kernel_size=(3, 3), stride=(1, 1), padding=same)
              (1): ReLU()
              (2): Conv2d(512, 512, kernel_size=(1, 1), stride=(1, 1), padding=same)
              (3): ReLU()
              (4): ZeroConv2d(
                (conv): Conv2d(512, 12, kernel_size=(3, 3), stride=(1, 1))
              )
            )
          )
        )
      )
    )
  